# 03. Linear Machine Learning Models

**Objective:** Implement and evaluate linear ML regression models for inflation forecasting

**Models:**
1. LASSO (Least Absolute Shrinkage and Selection Operator)
2. Ridge Regression
3. Elastic Net
4. LARS (Least Angle Regression)

**Evaluation Metrics:**
- RMSFE (Root Mean Squared Forecast Error)
- MAPE (Mean Absolute Percentage Error)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Linear ML models
from sklearn.linear_model import Lasso, Ridge, ElasticNet, Lars
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

# Custom metrics
import sys
sys.path.append('../')
from utils.metrics import rmsfe, mape, evaluate_model

# Paths
PROCESSED_DATA_PATH = Path('../data/processed')
RESULTS_PATH = Path('../results')
RESULTS_PATH.mkdir(parents=True, exist_ok=True)

## 1. Load Processed Data

In [ ]:
# Load train and test sets
df_train = pd.read_csv(PROCESSED_DATA_PATH / 'df_train.csv', index_col=0, parse_dates=True)
df_test = pd.read_csv(PROCESSED_DATA_PATH / 'df_test.csv', index_col=0, parse_dates=True)

print(f"Train set: {df_train.shape}")
print(f"Test set:  {df_test.shape}")

# Target variable (first column)
target_col = df_train.columns[0]
feature_cols = df_train.columns[1:] if len(df_train.columns) > 1 else df_train.columns[:1]

print(f"\nTarget: {target_col}")
print(f"Features: {list(feature_cols)}")

## 2. Prepare Data for ML Models

In [ ]:
# For univariate case, create lag features
def create_supervised_data(data, n_lags=12):
    """
    Transform time series into supervised learning format
    """
    df = pd.DataFrame()
    
    # Target (t)
    df['y'] = data.values
    
    # Lag features (t-1, t-2, ..., t-n)
    for i in range(1, n_lags + 1):
        df[f'lag_{i}'] = data.shift(i).values
    
    # Remove rows with NaN
    df = df.dropna()
    
    return df

# Create supervised dataset
n_lags = 12
df_train_supervised = create_supervised_data(df_train[target_col], n_lags=n_lags)
df_test_supervised = create_supervised_data(df_test[target_col], n_lags=n_lags)

# Split into X and y
X_train = df_train_supervised.drop('y', axis=1)
y_train = df_train_supervised['y']

X_test = df_test_supervised.drop('y', axis=1)
y_test = df_test_supervised['y']

print(f"\nX_train shape: {X_train.shape}")
print(f"X_test shape:  {X_test.shape}")

In [ ]:
# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("✓ Features standardized")

## 3. Model 1: LASSO Regression

LASSO performs L1 regularization, which can lead to sparse solutions (feature selection).

$$\min_{\beta} \|y - X\beta\|^2 + \alpha\|\beta\|_1$$

In [ ]:
# LASSO with hyperparameter tuning
lasso_params = {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0]}
lasso_grid = GridSearchCV(Lasso(max_iter=10000), lasso_params, cv=5, scoring='neg_mean_squared_error')
lasso_grid.fit(X_train_scaled, y_train)

print(f"Best alpha: {lasso_grid.best_params_['alpha']}")

# Best model
lasso_model = lasso_grid.best_estimator_
lasso_predictions = lasso_model.predict(X_test_scaled)

# Evaluate
lasso_results = evaluate_model(y_test.values, lasso_predictions, "LASSO")

## 4. Model 2: Ridge Regression

Ridge performs L2 regularization, which shrinks coefficients but keeps all features.

$$\min_{\beta} \|y - X\beta\|^2 + \alpha\|\beta\|^2_2$$

In [ ]:
# Ridge with hyperparameter tuning
ridge_params = {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}
ridge_grid = GridSearchCV(Ridge(), ridge_params, cv=5, scoring='neg_mean_squared_error')
ridge_grid.fit(X_train_scaled, y_train)

print(f"Best alpha: {ridge_grid.best_params_['alpha']}")

# Best model
ridge_model = ridge_grid.best_estimator_
ridge_predictions = ridge_model.predict(X_test_scaled)

# Evaluate
ridge_results = evaluate_model(y_test.values, ridge_predictions, "Ridge")

## 5. Model 3: Elastic Net

Elastic Net combines L1 and L2 regularization.

$$\min_{\beta} \|y - X\beta\|^2 + \alpha\rho\|\beta\|_1 + \frac{\alpha(1-\rho)}{2}\|\beta\|^2_2$$

In [ ]:
# Elastic Net with hyperparameter tuning
elasticnet_params = {
    'alpha': [0.001, 0.01, 0.1, 1.0],
    'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]
}
elasticnet_grid = GridSearchCV(ElasticNet(max_iter=10000), elasticnet_params, cv=5, scoring='neg_mean_squared_error')
elasticnet_grid.fit(X_train_scaled, y_train)

print(f"Best params: {elasticnet_grid.best_params_}")

# Best model
elasticnet_model = elasticnet_grid.best_estimator_
elasticnet_predictions = elasticnet_model.predict(X_test_scaled)

# Evaluate
elasticnet_results = evaluate_model(y_test.values, elasticnet_predictions, "Elastic Net")

## 6. Model 4: LARS (Least Angle Regression)

LARS is an efficient algorithm for fitting LASSO-type models.

In [ ]:
# LARS model
lars_model = Lars(n_nonzero_coefs=10)  # Limit number of features
lars_model.fit(X_train_scaled, y_train)

lars_predictions = lars_model.predict(X_test_scaled)

# Evaluate
lars_results = evaluate_model(y_test.values, lars_predictions, "LARS")

## 7. Visualize Results

In [ ]:
# Plot predictions vs actual
plt.figure(figsize=(14, 8))

# Get test indices
test_indices = df_test_supervised.index

plt.plot(range(len(y_test)), y_test.values, label='Actual', linewidth=2.5, marker='o', markersize=6)
plt.plot(range(len(y_test)), lasso_predictions, label='LASSO', linewidth=2, marker='s', alpha=0.7)
plt.plot(range(len(y_test)), ridge_predictions, label='Ridge', linewidth=2, marker='^', alpha=0.7)
plt.plot(range(len(y_test)), elasticnet_predictions, label='Elastic Net', linewidth=2, marker='d', alpha=0.7)
plt.plot(range(len(y_test)), lars_predictions, label='LARS', linewidth=2, marker='*', alpha=0.7)

plt.title('Linear ML Models - Forecasts vs Actual', fontsize=16)
plt.xlabel('Test Sample Index', fontsize=12)
plt.ylabel('Inflation Rate (%)', fontsize=12)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(RESULTS_PATH / 'figures' / 'linear_ml_forecasts.png', dpi=300, bbox_inches='tight')
plt.show()

## 8. Compare Model Performance

In [ ]:
# Compile results
results_list = [lasso_results, ridge_results, elasticnet_results, lars_results]
results_df = pd.DataFrame(results_list)
results_df = results_df.sort_values('RMSFE')

print("\n" + "="*50)
print("LINEAR ML MODELS PERFORMANCE SUMMARY")
print("="*50)
print(results_df.to_string(index=False))
print("="*50)

# Save results
results_df.to_csv(RESULTS_PATH / 'tables' / 'linear_ml_results.csv', index=False)
print("\n✓ Results saved to results/tables/linear_ml_results.csv")

In [ ]:
# Feature importance (coefficients)
feature_importance = pd.DataFrame({
    'Feature': X_train.columns,
    'LASSO': lasso_model.coef_,
    'Ridge': ridge_model.coef_,
    'Elastic Net': elasticnet_model.coef_,
    'LARS': lars_model.coef_
})

print("\nFeature Importance (Coefficients):")
print(feature_importance)

In [ ]:
# Save predictions
predictions_df = pd.DataFrame({
    'Actual': y_test.values,
    'LASSO': lasso_predictions,
    'Ridge': ridge_predictions,
    'Elastic_Net': elasticnet_predictions,
    'LARS': lars_predictions
})

predictions_df.to_csv(RESULTS_PATH / 'tables' / 'linear_ml_predictions.csv')
print("✓ Predictions saved to results/tables/linear_ml_predictions.csv")

## Summary

**Linear ML models completed:**
- ✓ LASSO Regression
- ✓ Ridge Regression
- ✓ Elastic Net
- ✓ LARS

**Next steps:**
- Run nonlinear ML models (Notebook 04)
- Compare all models (Notebook 05)